In [55]:
# import libraries and modules
import kagglehub
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime, timedelta
from kagglehub.datasets import KaggleDatasetAdapter
from fuzzywuzzy import process

In [ ]:
# importing Sackmann tennis match data - excluding 2025 as season has not finished yet

years = range(2000, 2024)  # 2000-2024
base_url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/"

dfs = []
for year in years:
    url = f"{base_url}atp_matches_{year}.csv"
    try:
        df = pd.read_csv(url)
        df['year'] = year 
        dfs.append(df)
    except Exception as e:
        print(f"Could not load {url}: {e}")

# Combine into one DataFrame
all_matches_df = pd.concat(dfs, ignore_index=True)

In [82]:
all_matches_df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,year
0,2000-301,Auckland,Hard,32,A,20000110,1,103163,1.0,NaN,...,39.0,29.0,17.0,4.0,7.0,11.0,1612.0,63.0,595.0,2000
1,2000-301,Auckland,Hard,32,A,20000110,2,102607,NaN,Q,...,25.0,18.0,12.0,3.0,6.0,211.0,157.0,49.0,723.0,2000
2,2000-301,Auckland,Hard,32,A,20000110,3,103252,NaN,NaN,...,20.0,7.0,8.0,7.0,11.0,48.0,726.0,59.0,649.0,2000
3,2000-301,Auckland,Hard,32,A,20000110,4,103507,7.0,NaN,...,29.0,14.0,10.0,6.0,8.0,45.0,768.0,61.0,616.0,2000
4,2000-301,Auckland,Hard,32,A,20000110,5,102103,NaN,Q,...,34.0,18.0,12.0,5.0,9.0,167.0,219.0,34.0,873.0,2000


In [83]:
all_matches_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71830 entries, 0 to 71829
Data columns (total 50 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tourney_id          71830 non-null  object 
 1   tourney_name        71830 non-null  object 
 2   surface             71777 non-null  object 
 3   draw_size           71830 non-null  int64  
 4   tourney_level       71830 non-null  object 
 5   tourney_date        71830 non-null  int64  
 6   match_num           71830 non-null  int64  
 7   winner_id           71830 non-null  int64  
 8   winner_seed         29826 non-null  float64
 9   winner_entry        9029 non-null   object 
 10  winner_name         71830 non-null  object 
 11  winner_hand         71830 non-null  object 
 12  winner_ht           70424 non-null  float64
 13  winner_ioc          71830 non-null  object 
 14  winner_age          71826 non-null  float64
 15  loser_id            71830 non-null  int64  
 16  lose

In [84]:
# Converting tourney_date to datetime
all_matches_df['tourney_date'] = pd.to_datetime(all_matches_df['tourney_date'].astype(str), format='%Y%m%d')

In [85]:
all_matches_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71830 entries, 0 to 71829
Data columns (total 50 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   tourney_id          71830 non-null  object        
 1   tourney_name        71830 non-null  object        
 2   surface             71777 non-null  object        
 3   draw_size           71830 non-null  int64         
 4   tourney_level       71830 non-null  object        
 5   tourney_date        71830 non-null  datetime64[ns]
 6   match_num           71830 non-null  int64         
 7   winner_id           71830 non-null  int64         
 8   winner_seed         29826 non-null  float64       
 9   winner_entry        9029 non-null   object        
 10  winner_name         71830 non-null  object        
 11  winner_hand         71830 non-null  object        
 12  winner_ht           70424 non-null  float64       
 13  winner_ioc          71830 non-null  object    

In [86]:
# Count NaN
all_matches_df.isna().sum()

tourney_id                0
tourney_name              0
surface                  53
draw_size                 0
tourney_level             0
tourney_date              0
match_num                 0
winner_id                 0
winner_seed           42004
winner_entry          62801
winner_name               0
winner_hand               0
winner_ht              1406
winner_ioc                0
winner_age                4
loser_id                  0
loser_seed            55349
loser_entry           57126
loser_name                0
loser_hand                4
loser_ht               2859
loser_ioc                 0
loser_age                 2
score                     0
best_of                   0
round                     0
minutes                7936
w_ace                  6460
w_df                   6460
w_svpt                 6460
w_1stIn                6460
w_1stWon               6460
w_2ndWon               6460
w_SvGms                6459
w_bpSaved              6460
w_bpFaced           

In [87]:
for col in all_matches_df.columns: 
    null_pct = all_matches_df[col].isnull().mean() * 100
    print(f"{col}: {null_pct:.2f}%")

tourney_id: 0.00%
tourney_name: 0.00%
surface: 0.07%
draw_size: 0.00%
tourney_level: 0.00%
tourney_date: 0.00%
match_num: 0.00%
winner_id: 0.00%
winner_seed: 58.48%
winner_entry: 87.43%
winner_name: 0.00%
winner_hand: 0.00%
winner_ht: 1.96%
winner_ioc: 0.00%
winner_age: 0.01%
loser_id: 0.00%
loser_seed: 77.06%
loser_entry: 79.53%
loser_name: 0.00%
loser_hand: 0.01%
loser_ht: 3.98%
loser_ioc: 0.00%
loser_age: 0.00%
score: 0.00%
best_of: 0.00%
round: 0.00%
minutes: 11.05%
w_ace: 8.99%
w_df: 8.99%
w_svpt: 8.99%
w_1stIn: 8.99%
w_1stWon: 8.99%
w_2ndWon: 8.99%
w_SvGms: 8.99%
w_bpSaved: 8.99%
w_bpFaced: 8.99%
l_ace: 8.99%
l_df: 8.99%
l_svpt: 8.99%
l_1stIn: 8.99%
l_1stWon: 8.99%
l_2ndWon: 8.99%
l_SvGms: 8.99%
l_bpSaved: 8.99%
l_bpFaced: 8.99%
winner_rank: 0.77%
winner_rank_points: 0.77%
loser_rank: 1.99%
loser_rank_points: 1.99%
year: 0.00%


In [63]:
# Import of data with betting odds from 2001 through 2025, odds data unavailable until 2001

all_years_df = pd.DataFrame()

for year in range(2001, 2025):
    try:
        df = pd.read_excel(f'tennis_data_co_uk_alldata/{year}.xls')
        print(f'Loaded {year}.xls')
    except FileNotFoundError:
        try:
            df = pd.read_excel(f'tennis_data_co_uk_alldata/{year}.xlsx')
            print(f'Loaded {year}.xlsx')
        except FileNotFoundError:
            print(f'No file found for year {year}')
            continue
    all_years_df = pd.concat([all_years_df, df], ignore_index=True)

Loaded 2001.xls
Loaded 2002.xls
Loaded 2003.xls
Loaded 2004.xls
Loaded 2005.xls
Loaded 2006.xls
Loaded 2007.xls
Loaded 2008.xls
Loaded 2009.xls
Loaded 2010.xls
Loaded 2011.xls
Loaded 2012.xls


/Users/jordanwork/Desktop/anaconda3/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


Loaded 2013.xlsx


/Users/jordanwork/Desktop/anaconda3/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


Loaded 2014.xlsx


/Users/jordanwork/Desktop/anaconda3/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


Loaded 2015.xlsx


/Users/jordanwork/Desktop/anaconda3/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


Loaded 2016.xlsx
Loaded 2017.xlsx
Loaded 2018.xlsx
Loaded 2019.xlsx
Loaded 2020.xlsx
Loaded 2021.xlsx
Loaded 2022.xlsx
Loaded 2023.xlsx
Loaded 2024.xlsx


In [64]:
all_years_df.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,UBW,UBL,LBW,LBL,SJW,SJL,MaxW,MaxL,AvgW,AvgL
0,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3.0,Clement A.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3.0,Goldstein P.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3.0,Haas T.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3.0,Henman T.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3.0,Hewitt L.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
all_years_df['Date'] = pd.to_datetime(all_years_df['Date'].astype(str), format='%Y-%m-%d')

In [66]:
all_years_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63739 entries, 0 to 63738
Data columns (total 54 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ATP         63739 non-null  int64         
 1   Location    63739 non-null  object        
 2   Tournament  63739 non-null  object        
 3   Date        63739 non-null  datetime64[ns]
 4   Series      63739 non-null  object        
 5   Court       63739 non-null  object        
 6   Surface     63739 non-null  object        
 7   Round       63739 non-null  object        
 8   Best of     63724 non-null  float64       
 9   Winner      63739 non-null  object        
 10  Loser       63739 non-null  object        
 11  WRank       63717 non-null  float64       
 12  LRank       63625 non-null  object        
 13  W1          63387 non-null  float64       
 14  L1          63390 non-null  float64       
 15  W2          62805 non-null  object        
 16  L2          62806 non-

In [67]:
# Creating sets to find overlap in the two 
set_tournament = set(all_years_df['Tournament'].dropna().unique())
set_tourney_name = set(all_matches_df['tourney_name'].dropna().unique())

# Find the intersection
overlap = set_tournament.intersection(set_tourney_name)

print(f"Number of overlapping tournaments: {len(overlap)}")
print("Some overlapping values:", list(overlap)[:10])

Number of overlapping tournaments: 9
Some overlapping values: ['Great Ocean Road Open', 'Australian Open', 'Masters Cup', 'Monte Carlo Masters', 'Murray River Open', 'Shanghai Masters', 'US Open', 'Madrid Masters', 'Wimbledon']


In [68]:
# Clean column names

def clean_names(series):
    return (series.str.lower()                    # lowercase
                  .str.strip()                    # remove leading/trailing spaces
                  .str.replace(r'[^a-z0-9 ]', '', regex=True)  # remove punctuation/special chars
                  .str.replace(r'\s+', ' ', regex=True)       # collapse multiple spaces
           )

all_years_df['Tournament_clean'] = clean_names(all_years_df['Tournament'].astype(str))
all_matches_df['tourney_name_clean'] = clean_names(all_matches_df['tourney_name'].astype(str))

In [69]:
# Get unique cleaned tournament names from all_matches_df
tourneys = all_matches_df['tourney_name_clean'].dropna().unique()

def find_best_match(name, choices, threshold=90):
    match, score = process.extractOne(name, choices)
    if score >= threshold:
        return match
    else:
        return None

# mapping from all_years_df cleaned names to best matches in all_matches_df
mapping = {}
for name in all_years_df['Tournament_clean'].dropna().unique():
    best_match = find_best_match(name, tourneys)
    if best_match:
        mapping[name] = best_match

print(mapping)

{'australian open': 'australian open', 'milan indoors': 'milan', 'copenhagen open': 'copenhagen', 'marseille open': 'marseille', 'dubai championships': 'dubai', 'estoril open': 'estoril', 'monte carlo masters': 'monte carlo masters', 'verizon tennis challenge': 'halle', 'mallorca open': 'mallorca', 'hamburg tms': 'hamburg', 'nottingham open': 'nottingham', 'wimbledon': 'wimbledon', 'gstaad open': 'gstaad', 'us open': 'us open', 'grand prix de lyon': 'lyon', 'stuttgart tms': 'stuttgart', 'st petersburg open': 'st petersburg', 'stockholm open': 'stockholm', 'masters cup': 'masters cup', 'queens club': 'queens club', 'madrid masters': 'madrid masters', 'cam open comunidad valenciana': 'valencia', 'indesit atp milano indoor': 'milan', 'atp buenos aires 2004': 'buenos aires', 'atp buenos aires 2005': 'buenos aires', 'dubai duty free mens open': 'dubai', 'open de tenis comunidad valenciana': 'valencia', 'next generation adelaide international': 'adelaide', 'chennai open': 'chennai', 'sydney 

In [70]:
# Reverse the mapping so we can join on the official tournament name

reverse_mapping = {v: k for k, v in mapping.items()}
all_matches_df['mapped_tourney_name'] = all_matches_df['tourney_name_clean'].map(reverse_mapping)

In [71]:
all_years_df['year'] = all_years_df['Date'].dt.year

In [72]:
# Standardizing player names for matching: 

def clean_player_names(series):
    return (series.str.lower()
                  .str.strip()
                  .str.replace(r'[^a-z ]', '', regex=True)
                  .str.replace(r'\s+', ' ', regex=True)
           )

all_years_df['Winner_clean'] = clean_player_names(all_years_df['Winner'].astype(str))
all_years_df['Loser_clean'] = clean_player_names(all_years_df['Loser'].astype(str))

all_matches_df['winner_name_clean'] = clean_player_names(all_matches_df['winner_name'].astype(str))
all_matches_df['loser_name_clean'] = clean_player_names(all_matches_df['loser_name'].astype(str))

In [73]:
years_sub = all_years_df[['Tournament_clean', 'year', 'Winner_clean', 'Loser_clean', 'B365W', 'B365L']]

In [74]:
merged_df = all_matches_df.merge(years_sub,left_on=['mapped_tourney_name', 'year', 'winner_name_clean', 'loser_name_clean'],
                                 right_on=['Tournament_clean', 'year', 'Winner_clean', 'Loser_clean'], how='left')

In [75]:
merged_df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,year,tourney_name_clean,mapped_tourney_name,winner_name_clean,loser_name_clean,Tournament_clean,Winner_clean,Loser_clean,B365W,B365L
0,2000-301,Auckland,Hard,32,A,2000-01-10,1,103163,1.0,NaN,...,2000,auckland,NaN,tommy haas,jeff tarango,NaN,NaN,NaN,NaN,NaN
1,2000-301,Auckland,Hard,32,A,2000-01-10,2,102607,NaN,Q,...,2000,auckland,NaN,juan balcells,franco squillari,NaN,NaN,NaN,NaN,NaN
2,2000-301,Auckland,Hard,32,A,2000-01-10,3,103252,NaN,NaN,...,2000,auckland,NaN,alberto martin,alberto berasategui,NaN,NaN,NaN,NaN,NaN
3,2000-301,Auckland,Hard,32,A,2000-01-10,4,103507,7.0,NaN,...,2000,auckland,NaN,juan carlos ferrero,roger federer,NaN,NaN,NaN,NaN,NaN
4,2000-301,Auckland,Hard,32,A,2000-01-10,5,102103,NaN,Q,...,2000,auckland,NaN,michael sell,nicolas escude,NaN,NaN,NaN,NaN,NaN


In [76]:
merged_df.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points',
       'year', 'tourney_name_clean', 'mapped_tourney_name',
       'winner_name_clean', 'loser_name_clean', 'Tournament_clean',
       'Winner_clean', 'Loser_clean', 'B365W', 'B365L'],
      dtype='object')

In [77]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71830 entries, 0 to 71829
Data columns (total 59 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   tourney_id           71830 non-null  object        
 1   tourney_name         71830 non-null  object        
 2   surface              71777 non-null  object        
 3   draw_size            71830 non-null  int64         
 4   tourney_level        71830 non-null  object        
 5   tourney_date         71830 non-null  datetime64[ns]
 6   match_num            71830 non-null  int64         
 7   winner_id            71830 non-null  int64         
 8   winner_seed          29826 non-null  float64       
 9   winner_entry         9029 non-null   object        
 10  winner_name          71830 non-null  object        
 11  winner_hand          71830 non-null  object        
 12  winner_ht            70424 non-null  float64       
 13  winner_ioc           71830 non-